In [107]:
import pandas as pd
from tensorflow.keras import models
import time, copy
import numpy as np

In [95]:
df=pd.read_csv('test_mydata.csv')
df_test=df.copy()
df_test=df_test.drop(['id','sat_id'], axis=1)
df_test.head()

,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
1,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237
2,17281.246143,-2011.121378,39970.739873,-0.104355,2.653343,-0.295245,17241.395285,-2016.770194,39958.759155,-0.103839,2.655338,-0.294835
3,-1447.724638,28032.228987,-3918.192838,-1.560091,-1.321435,-3.577772,-1461.093061,28026.085525,-3959.767549,-1.557072,-1.324386,-3.577769
4,-10583.521489,-10145.003880,-24243.775722,0.275212,-4.049127,0.719273,-10574.016380,-10147.909472,-24270.149411,0.274385,-4.046727,0.718858


In [96]:
df_stats=pd.read_csv('train_stats.csv', index_col=0)
df_stats=df_stats.drop('sat_id')
df_stats.head()

,count,mean,std,min,25%,50%,75%,max
x_sim,519930.0,-941.331006,25387.037253,-255037.952966,-13139.771199,-612.877803,11194.372299,200961.158000
y_sim,519930.0,319.853163,25603.245579,-281784.430040,-11758.892200,501.609913,12558.620911,209619.553857
z_sim,519930.0,-77.075471,20670.833347,-180250.922287,-7963.461284,57.093264,8253.028193,198526.091552
Vx_sim,519930.0,-0.001115,2.636436,-8.240694,-1.896241,0.019026,1.954861,7.431844
Vy_sim,519930.0,-0.002132,2.599643,-8.348831,-1.878077,-0.018243,1.857399,8.021438


In [97]:
label=['x','y','z','Vx', 'Vy', 'Vz']
df_label=df_test[label]
df_test=df_test.drop(label, axis=1)
df_test.head()

,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
1,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237
2,17241.395285,-2016.770194,39958.759155,-0.103839,2.655338,-0.294835
3,-1461.093061,28026.085525,-3959.767549,-1.557072,-1.324386,-3.577769
4,-10574.016380,-10147.909472,-24270.149411,0.274385,-4.046727,0.718858


In [98]:
def norm(x):
  try:
    return (x - df_stats['mean']) / df_stats['std']
  except Exception as e:
    print(e)

In [99]:
normed_test_data = norm(df_test)
normed_test_data.head()

,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,-0.378704,0.051925,-1.180214,-0.114772,-1.641642,-0.291518
1,-0.227991,-1.141345,-0.721564,0.375696,-0.969551,1.107991
2,0.716221,-0.091263,1.936827,-0.038963,1.022244,-0.139380
3,-0.020474,1.082138,-0.187834,-0.590174,-0.508629,-1.692253
4,-0.379433,-0.408845,-1.170397,0.104497,-1.555827,0.340110


In [100]:
list_variable={'x':['x_sim', 'z_sim', 'Vy_sim'],
         'y': ['x_sim', 'y_sim', 'Vz_sim'],
         'z': ['x_sim', 'z_sim', 'Vy_sim', 'Vz_sim'],
         'Vx': ['y_sim', 'Vx_sim', 'Vz_sim'],
         'Vy': ['z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'],
         'Vz': ['y_sim', 'z_sim', 'Vy_sim', 'Vz_sim']}
list_variable

{'x': ['x_sim', 'z_sim', 'Vy_sim'],
 'y': ['x_sim', 'y_sim', 'Vz_sim'],
 'z': ['x_sim', 'z_sim', 'Vy_sim', 'Vz_sim'],
 'Vx': ['y_sim', 'Vx_sim', 'Vz_sim'],
 'Vy': ['z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'],
 'Vz': ['y_sim', 'z_sim', 'Vy_sim', 'Vz_sim']}

In [101]:
for i in list_variable:
    model=models.load_model('model/8_model_'+i+'.h5')
    df[i+'_pred']=model.predict(normed_test_data[list_variable[i]].values)

In [102]:
df.head()

,id,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,x_pred,y_pred,z_pred,Vx_pred,Vy_pred,Vz_pred
0,1,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468,-10622.737305,1645.848999,-23786.261719,-0.266338,-4.060851,-1.981395
1,4,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237,-9240.502930,-28843.664062,-15672.079102,0.995836,-2.507833,2.264112
2,15,0,17281.246143,-2011.121378,39970.739873,-0.104355,2.653343,-0.295245,17241.395285,-2016.770194,39958.759155,-0.103839,2.655338,-0.294835,17211.457031,-5968.889648,39868.957031,-0.074913,2.688053,0.639208
3,22,0,-1447.724638,28032.228987,-3918.192838,-1.560091,-1.321435,-3.577772,-1461.093061,28026.085525,-3959.767549,-1.557072,-1.324386,-3.577769,4235.652344,28029.210938,6307.504883,-1.458712,0.364314,-3.557481
4,26,0,-10583.521489,-10145.003880,-24243.775722,0.275212,-4.049127,0.719273,-10574.016380,-10147.909472,-24270.149411,0.274385,-4.046727,0.718858,-10580.450195,-7954.033203,-24218.878906,0.302837,-3.921396,-0.337146


In [105]:
def smape(satellite_predicted_values, satellite_true_values):
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values)/(np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [108]:
for i in ['_sim', '_pred']:
    real=df[label].values
    comp=df[[j+i for j in label]].values
    print('smape real vs %s : %f'%(i, smape(comp, real)))

smape real vs _sim : 0.187985
smape real vs _pred : 0.145908
